# LangChain Expression Language (LCEL) Basics

## LCEL là gì?

**LangChain Expression Language (LCEL)** là ngôn ngữ biểu thức declarative để compose chains một cách dễ dàng và hiệu quả. LCEL được thiết kế để:

### 🎯 **Lợi ích chính của LCEL**

1. **🔄 Streaming Support**: Automatic streaming của intermediate steps
2. **⚡ Async Support**: Native async/await support
3. **🔍 Optimized Parallel Execution**: Tự động tối ưu parallel operations
4. **🛠️ Retries và Fallbacks**: Built-in retry logic và fallback mechanisms
5. **📊 Traceability**: Detailed tracing và debugging information
6. **🔧 Flexibility**: Easy composition và modification

### 🔗 **Pipe Operator (`|`)**
LCEL sử dụng pipe operator để chain components:
```python
chain = component1 | component2 | component3
```

Điều này có nghĩa:
- Output của `component1` becomes input của `component2`
- Output của `component2` becomes input của `component3`
- Kết quả cuối cùng là output của `component3`

### 🧩 **Core Runnable Components**
- **RunnablePassthrough**: Pass input through unchanged
- **RunnableParallel**: Execute multiple runnables in parallel
- **RunnableLambda**: Wrap functions as runnables
- **RunnableBranch**: Conditional execution

## Setup và Import

In [ ]:
# Import các thư viện cần thiết
import os
import asyncio
import time
from dotenv import load_dotenv
from typing import Dict, List, Any

# LangChain Core
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
    RunnableBranch,
    Runnable
)

# LangChain Anthropic
from langchain_anthropic import ChatAnthropic

# Pydantic cho structured output
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# Load environment variables
load_dotenv()

print("✅ Đã import tất cả dependencies")

In [ ]:
# Khởi tạo ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.7,
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

# Test LLM
test_response = llm.invoke("Chào bạn! LCEL là gì?")
print("✅ LLM ready")
print(f"Test response: {test_response.content[:100]}...")

## 1. LCEL Syntax Basics - Pipe Operator

In [ ]:
# LCEL basic chain
prompt = ChatPromptTemplate.from_template(
    "Viết một bài thơ ngắn về {topic}. Style: {style}"
)
parser = StrOutputParser()

# LCEL chain với pipe operator
lcel_chain = prompt | llm | parser

print("✅ LCEL chain created: prompt | llm | parser")
print(f"Chain type: {type(lcel_chain)}")

# Test basic chain
result = lcel_chain.invoke({
    "topic": "mưa thu", 
    "style": "trữ tình, nhẹ nhàng"
})

print(f"\n📝 Poem result:")
print(result)

In [ ]:
# So sánh LCEL với traditional approach
def traditional_chain(input_data):
    """Traditional way without LCEL"""
    # Step 1: Format prompt
    formatted_prompt = prompt.format_messages(**input_data)
    
    # Step 2: Call LLM
    llm_response = llm.invoke(formatted_prompt)
    
    # Step 3: Parse output
    parsed_result = parser.parse(llm_response.content)
    
    return parsed_result

# Test both approaches
test_input = {"topic": "núi rừng", "style": "hùng tráng, mạnh mẽ"}

print("=== Comparison: LCEL vs Traditional ===")

# LCEL approach
start_time = time.time()
lcel_result = lcel_chain.invoke(test_input)
lcel_time = time.time() - start_time

print(f"\n🔗 LCEL Result ({lcel_time:.2f}s):")
print(lcel_result[:150] + "...")

# Traditional approach
start_time = time.time()
traditional_result = traditional_chain(test_input)
traditional_time = time.time() - start_time

print(f"\n🔧 Traditional Result ({traditional_time:.2f}s):")
print(traditional_result[:150] + "...")

print(f"\n⚡ Performance: LCEL có thể optimize parallel operations internally")

## 2. RunnablePassthrough - Pass Through Values

In [ ]:
# RunnablePassthrough để preserve original input
from langchain_core.runnables import RunnablePassthrough

# Simple passthrough example
print("=== RunnablePassthrough Basics ===")

# Test passthrough
passthrough = RunnablePassthrough()
test_data = {"name": "Alice", "age": 25, "city": "Hanoi"}

passthrough_result = passthrough.invoke(test_data)
print(f"Input: {test_data}")
print(f"Passthrough output: {passthrough_result}")
print(f"Same object? {test_data is passthrough_result}")

In [ ]:
# RunnablePassthrough.assign để add computed values
def compute_description(data):
    """Compute description từ input data"""
    return f"{data['name']} là người {data['age']} tuổi sống tại {data['city']}"

def compute_category(data):
    """Compute age category"""
    age = data['age']
    if age < 18:
        return "Trẻ em"
    elif age < 65:
        return "Người lớn"
    else:
        return "Người cao tuổi"

# Chain với RunnablePassthrough.assign
enrich_data_chain = RunnablePassthrough.assign(
    description=RunnableLambda(compute_description),
    category=RunnableLambda(compute_category)
)

print("\n=== RunnablePassthrough.assign ===")

# Test assign
enriched_result = enrich_data_chain.invoke(test_data)
print(f"Original data: {test_data}")
print(f"Enriched data: {enriched_result}")

# Show what was added
print(f"\n📝 Added fields:")
print(f"- description: {enriched_result['description']}")
print(f"- category: {enriched_result['category']}")

In [ ]:
# Use case: Preserve context trong LLM chain
context_prompt = ChatPromptTemplate.from_template(
    """Dựa trên thông tin sau, viết một introduction ngắn:
    
    Tên: {name}
    Tuổi: {age}
    Thành phố: {city}
    Mô tả: {description}
    Nhóm tuổi: {category}
    
    Introduction:"""
)

# Chain preserves input + adds LLM output
context_preserving_chain = (
    RunnablePassthrough.assign(
        description=RunnableLambda(compute_description),
        category=RunnableLambda(compute_category)
    )
    | RunnablePassthrough.assign(
        introduction=context_prompt | llm | StrOutputParser()
    )
)

print("\n=== Context Preserving Chain ===")

# Test với multiple people
people = [
    {"name": "Minh", "age": 28, "city": "Ho Chi Minh"},
    {"name": "Lan", "age": 35, "city": "Da Nang"},
    {"name": "Duc", "age": 45, "city": "Hanoi"}
]

for person in people:
    result = context_preserving_chain.invoke(person)
    print(f"\n👤 {person['name']}:")
    print(f"📋 Category: {result['category']}")
    print(f"📝 Introduction: {result['introduction']}")
    print("-" * 50)

## 3. RunnableParallel - Parallel Execution

In [ ]:
# RunnableParallel để execute multiple runnables cùng lúc
from langchain_core.runnables import RunnableParallel

# Different analysis prompts
sentiment_prompt = ChatPromptTemplate.from_template(
    "Phân tích cảm xúc của đoạn text sau (Positive/Negative/Neutral): {text}"
)

summary_prompt = ChatPromptTemplate.from_template(
    "Tóm tắt đoạn text sau trong 1 câu: {text}"
)

keywords_prompt = ChatPromptTemplate.from_template(
    "Trích xuất 3 từ khóa chính từ đoạn text sau: {text}"
)

# Parallel analysis chain
parallel_analysis = RunnableParallel(
    sentiment=sentiment_prompt | llm | StrOutputParser(),
    summary=summary_prompt | llm | StrOutputParser(),
    keywords=keywords_prompt | llm | StrOutputParser(),
    original=RunnablePassthrough()
)

print("✅ Parallel analysis chain created")
print("Components: sentiment, summary, keywords, original")

In [ ]:
# Test parallel execution
sample_texts = [
    "Hôm nay trời đẹp quá! Tôi rất vui vì được đi dạo công viên với gia đình. Không khí trong lành và mọi người đều tươi cười.",
    "Dự án này gặp nhiều khó khăn. Timeline bị delay và budget vượt quá dự kiến. Team đang stress và cần support.",
    "Báo cáo hôm nay cho thấy doanh số tăng 15% so với quý trước. Sản phẩm mới được khách hàng đón nhận tích cực."
]

print("=== Parallel Analysis Results ===")

for i, text in enumerate(sample_texts, 1):
    print(f"\n📄 Text {i}: {text[:80]}...")
    
    # Measure time for parallel execution
    start_time = time.time()
    result = parallel_analysis.invoke({"text": text})
    execution_time = time.time() - start_time
    
    print(f"⏱️ Execution time: {execution_time:.2f}s")
    print(f"😊 Sentiment: {result['sentiment']}")
    print(f"📝 Summary: {result['summary']}")
    print(f"🔑 Keywords: {result['keywords']}")
    print("-" * 70)

In [ ]:
# So sánh parallel vs sequential execution
def sequential_analysis(text_input):
    """Sequential execution cho comparison"""
    text = text_input["text"]
    
    # Execute each analysis sequentially
    sentiment_chain = sentiment_prompt | llm | StrOutputParser()
    summary_chain = summary_prompt | llm | StrOutputParser()
    keywords_chain = keywords_prompt | llm | StrOutputParser()
    
    return {
        "sentiment": sentiment_chain.invoke({"text": text}),
        "summary": summary_chain.invoke({"text": text}),
        "keywords": keywords_chain.invoke({"text": text}),
        "original": text_input
    }

# Benchmark comparison
test_text = {"text": sample_texts[0]}

print("\n=== Performance Comparison ===")

# Parallel execution
start_time = time.time()
parallel_result = parallel_analysis.invoke(test_text)
parallel_time = time.time() - start_time

print(f"🔄 Parallel execution: {parallel_time:.2f}s")

# Sequential execution
start_time = time.time()
sequential_result = sequential_analysis(test_text)
sequential_time = time.time() - start_time

print(f"⏩ Sequential execution: {sequential_time:.2f}s")
print(f"⚡ Speedup: {sequential_time/parallel_time:.1f}x faster với parallel")

# Verify results are similar
print(f"\n✅ Results comparison:")
print(f"Parallel sentiment: {parallel_result['sentiment']}")
print(f"Sequential sentiment: {sequential_result['sentiment']}")

## 4. Complex LCEL Compositions

In [ ]:
# Complex chain: preprocessing + parallel analysis + postprocessing
def preprocess_text(input_data):
    """Preprocess input text"""
    text = input_data["text"]
    
    # Basic preprocessing
    cleaned_text = text.strip()
    word_count = len(cleaned_text.split())
    char_count = len(cleaned_text)
    
    return {
        "text": cleaned_text,
        "word_count": word_count,
        "char_count": char_count,
        "is_long": word_count > 50
    }

def postprocess_results(parallel_results):
    """Combine và process parallel results"""
    # Extract components
    preprocessed = parallel_results["preprocessed"]
    analysis = parallel_results["analysis"]
    
    # Create comprehensive report
    report = {
        "text_stats": {
            "word_count": preprocessed["word_count"],
            "char_count": preprocessed["char_count"],
            "is_long": preprocessed["is_long"]
        },
        "analysis": analysis,
        "confidence_score": 0.95 if not preprocessed["is_long"] else 0.85
    }
    
    return report

# Complex composed chain
complex_chain = (
    # Step 1: Parallel preprocessing và analysis
    RunnableParallel(
        preprocessed=RunnableLambda(preprocess_text),
        analysis=parallel_analysis
    )
    # Step 2: Postprocess results
    | RunnableLambda(postprocess_results)
)

print("✅ Complex composed chain created")
print("Structure: parallel(preprocess, analysis) | postprocess")

In [ ]:
# Test complex chain
complex_test_cases = [
    {
        "text": "AI tuyệt vời!",  # Short text
        "name": "Short positive"
    },
    {
        "text": """Trí tuệ nhân tạo đang thay đổi cách chúng ta làm việc và sống. 
        Từ việc tự động hóa các tác vụ đơn giản đến hỗ trợ ra quyết định phức tạp, 
        AI mang lại nhiều cơ hội nhưng cũng đặt ra những thách thức về đạo đức 
        và tác động xã hội mà chúng ta cần xem xét cẩn thận.""",  # Long text
        "name": "Long complex"
    }
]

print("=== Complex Chain Results ===")

for test_case in complex_test_cases:
    print(f"\n📋 Test: {test_case['name']}")
    print(f"Text: {test_case['text'][:100]}...")
    
    result = complex_chain.invoke(test_case)
    
    # Display results
    stats = result["text_stats"]
    analysis = result["analysis"]
    
    print(f"\n📊 Text Statistics:")
    print(f"   Words: {stats['word_count']}, Chars: {stats['char_count']}")
    print(f"   Is long: {stats['is_long']}")
    print(f"   Confidence: {result['confidence_score']}")
    
    print(f"\n🔍 Analysis:")
    print(f"   Sentiment: {analysis['sentiment']}")
    print(f"   Summary: {analysis['summary']}")
    print(f"   Keywords: {analysis['keywords']}")
    
    print("-" * 70)

## 5. RunnableBranch - Conditional Execution

In [ ]:
# RunnableBranch để conditional execution
from langchain_core.runnables import RunnableBranch

# Different prompts cho different content types
technical_prompt = ChatPromptTemplate.from_template(
    """Phân tích kỹ thuật cho đoạn text sau:
    {text}
    
    Tập trung vào: terminologies, complexity, technical accuracy."""
)

creative_prompt = ChatPromptTemplate.from_template(
    """Phân tích sáng tạo cho đoạn text sau:
    {text}
    
    Tập trung vào: imagery, emotions, artistic expression."""
)

general_prompt = ChatPromptTemplate.from_template(
    """Phân tích tổng quát cho đoạn text sau:
    {text}
    
    Cung cấp analysis cân bằng về content và style."""
)

# Content type detection function
def detect_content_type(input_data):
    """Detect content type từ text"""
    text = input_data["text"].lower()
    
    # Simple keyword-based detection
    technical_keywords = ["algorithm", "api", "database", "framework", "technology", "system", "software", "ai", "machine learning"]
    creative_keywords = ["thơ", "poem", "story", "cảm xúc", "tình yêu", "mơ ước", "hoài niệm"]
    
    technical_score = sum(1 for keyword in technical_keywords if keyword in text)
    creative_score = sum(1 for keyword in creative_keywords if keyword in text)
    
    if technical_score > creative_score and technical_score > 0:
        return "technical"
    elif creative_score > 0:
        return "creative"
    else:
        return "general"

# Conditional chain với RunnableBranch
def create_conditional_chain():
    return RunnableBranch(
        # (condition, runnable) pairs
        (lambda x: detect_content_type(x) == "technical", technical_prompt | llm | StrOutputParser()),
        (lambda x: detect_content_type(x) == "creative", creative_prompt | llm | StrOutputParser()),
        # Default case
        general_prompt | llm | StrOutputParser()
    )

conditional_chain = create_conditional_chain()

print("✅ Conditional chain với RunnableBranch created")
print("Branches: technical, creative, general")

In [ ]:
# Test conditional execution
test_contents = [
    {
        "text": "Machine learning algorithms require careful tuning of hyperparameters to optimize model performance. The API design should follow RESTful principles.",
        "expected_type": "technical"
    },
    {
        "text": "Mưa thu rơi lặng lẽ trên phố phường, mang theo những hoài niệm xa xôi. Tình yêu như cánh bướm mong manh.",
        "expected_type": "creative"
    },
    {
        "text": "Hôm nay tôi đi mua sắm ở siêu thị. Có nhiều sản phẩm mới và giá cả hợp lý.",
        "expected_type": "general"
    }
]

print("=== Conditional Chain Testing ===")

for i, content in enumerate(test_contents, 1):
    detected_type = detect_content_type(content)
    
    print(f"\n📄 Content {i}:")
    print(f"Text: {content['text'][:80]}...")
    print(f"Expected type: {content['expected_type']}")
    print(f"Detected type: {detected_type}")
    print(f"Match: {'✅' if detected_type == content['expected_type'] else '❌'}")
    
    # Execute conditional chain
    result = conditional_chain.invoke(content)
    print(f"\n🔍 Analysis result:")
    print(f"{result[:200]}...")
    
    print("-" * 70)

## 6. Async Support trong LCEL

In [ ]:
# LCEL automatic async support
import asyncio
from typing import List

# Create async-compatible chain
async_prompt = ChatPromptTemplate.from_template(
    "Viết một fact thú vị về {topic}. Giữ ngắn gọn và hấp dẫn."
)

async_chain = async_prompt | llm | StrOutputParser()

print("✅ Async chain created")

# Async batch processing
async def process_topics_async(topics: List[str]):
    """Process multiple topics asynchronously"""
    print(f"🚀 Processing {len(topics)} topics asynchronously...")
    
    # Create inputs
    inputs = [{"topic": topic} for topic in topics]
    
    # Use abatch cho async batch processing
    start_time = time.time()
    results = await async_chain.abatch(inputs)
    end_time = time.time()
    
    print(f"⏱️ Async batch completed in {end_time - start_time:.2f}s")
    return results

# Sequential processing để compare
def process_topics_sync(topics: List[str]):
    """Process topics sequentially"""
    print(f"🐌 Processing {len(topics)} topics sequentially...")
    
    start_time = time.time()
    results = []
    for topic in topics:
        result = async_chain.invoke({"topic": topic})
        results.append(result)
    end_time = time.time()
    
    print(f"⏱️ Sequential processing completed in {end_time - start_time:.2f}s")
    return results

# Test topics
test_topics = ["vũ trụ", "đại dương", "AI", "lịch sử", "âm nhạc"]

print(f"\n=== Async vs Sequential Comparison ===")
print(f"Topics: {test_topics}")

In [ ]:
# Run async comparison
# Note: In Jupyter, we need to handle the event loop properly

try:
    # Try to get existing event loop
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # In Jupyter, create a new task
        import nest_asyncio
        nest_asyncio.apply()
        
        # Run async processing
        async_results = await process_topics_async(test_topics)
    else:
        # Run in new event loop
        async_results = asyncio.run(process_topics_async(test_topics))
except:
    # Fallback: simulate async với batch
    print("🔄 Using batch processing as async simulation...")
    inputs = [{"topic": topic} for topic in test_topics]
    
    start_time = time.time()
    async_results = async_chain.batch(inputs)
    async_time = time.time() - start_time
    print(f"⏱️ Batch processing completed in {async_time:.2f}s")

# Sequential processing
sync_results = process_topics_sync(test_topics)

# Display results
print(f"\n📋 Results comparison:")
for i, (topic, async_result, sync_result) in enumerate(zip(test_topics, async_results, sync_results)):
    print(f"\n{i+1}. {topic}:")
    print(f"   Async: {async_result[:100]}...")
    print(f"   Sync:  {sync_result[:100]}...")
    print(f"   Same: {'✅' if async_result == sync_result else '❌'}")

## 7. Streaming Support

In [ ]:
# LCEL streaming support
streaming_prompt = ChatPromptTemplate.from_template(
    "Viết một câu chuyện ngắn về {theme}. Kể chi tiết và sinh động."
)

streaming_chain = streaming_prompt | llm | StrOutputParser()

print("=== Streaming Example ===")
print("📖 Story theme: 'cuộc phiêu lưu trong rừng'")
print("\n🔄 Streaming output:")
print("-" * 50)

# Stream the response
full_response = ""
for chunk in streaming_chain.stream({"theme": "cuộc phiêu lưu trong rừng"}):
    print(chunk, end="", flush=True)
    full_response += chunk

print("\n" + "-" * 50)
print(f"✅ Streaming completed. Total length: {len(full_response)} characters")

In [ ]:
# Streaming với intermediate steps
intermediate_streaming_chain = (
    RunnablePassthrough.assign(
        processed_theme=RunnableLambda(lambda x: f"Enhanced theme: {x['theme']} với magic elements")
    )
    | ChatPromptTemplate.from_template(
        "Viết story về: {processed_theme}. Make it exciting!"
    )
    | llm
    | StrOutputParser()
)

print("\n=== Intermediate Streaming ===")
print("🎭 Theme: 'robot thông minh'")
print("\n📝 Story với intermediate processing:")
print("-" * 50)

# Stream với intermediate steps
for chunk in intermediate_streaming_chain.stream({"theme": "robot thông minh"}):
    print(chunk, end="", flush=True)

print("\n" + "-" * 50)
print("✅ Intermediate streaming completed")

## 8. Error Handling và Fallbacks

In [ ]:
# LCEL error handling với fallbacks
from langchain_core.runnables import RunnableWithFallbacks

# Primary chain có thể fail
def risky_processing(input_data):
    """Function có thể fail based on input"""
    text = input_data.get("text", "")
    
    # Simulate failure cho certain inputs
    if "error" in text.lower():
        raise ValueError("Processing failed: error keyword detected")
    
    if len(text) > 200:
        raise ValueError("Text too long for processing")
    
    return {"processed_text": f"Successfully processed: {text}"}

def safe_fallback(input_data):
    """Fallback processing"""
    return {"processed_text": f"Fallback processing: {input_data.get('text', 'No text')[:50]}..."}

# Primary và fallback chains
primary_chain = RunnableLambda(risky_processing)
fallback_chain = RunnableLambda(safe_fallback)

# Chain với fallback
robust_chain = primary_chain.with_fallbacks([fallback_chain])

print("✅ Robust chain với fallback created")
print("Primary: risky_processing, Fallback: safe_fallback")

In [ ]:
# Test error handling
test_cases_error = [
    {"text": "Normal text for processing", "should_fail": False},
    {"text": "This will cause an error", "should_fail": True},
    {"text": "A" * 250, "should_fail": True},  # Too long
    {"text": "Short safe text", "should_fail": False}
]

print("=== Error Handling Testing ===")

for i, test_case in enumerate(test_cases_error, 1):
    text_preview = test_case["text"][:50] + ("..." if len(test_case["text"]) > 50 else "")
    
    print(f"\n🧪 Test {i}: {text_preview}")
    print(f"Expected to fail: {test_case['should_fail']}")
    
    try:
        # Test primary chain alone
        primary_result = primary_chain.invoke(test_case)
        print(f"✅ Primary succeeded: {primary_result['processed_text'][:80]}...")
    except Exception as e:
        print(f"❌ Primary failed: {str(e)}")
    
    # Test robust chain với fallback
    try:
        robust_result = robust_chain.invoke(test_case)
        print(f"🛡️ Robust result: {robust_result['processed_text'][:80]}...")
    except Exception as e:
        print(f"💥 Even fallback failed: {str(e)}")
    
    print("-" * 60)

## 9. Advanced LCEL Patterns

In [ ]:
# Advanced pattern: Dynamic chain construction
def create_dynamic_chain(analysis_type: str, complexity: str):
    """Dynamically create chain based on parameters"""
    
    # Base components
    base_prompt = "Phân tích {type} cho text: {text}"
    
    if complexity == "simple":
        instruction = "Giữ analysis ngắn gọn và dễ hiểu."
    elif complexity == "detailed":
        instruction = "Cung cấp analysis chi tiết và sâu sắc."
    else:
        instruction = "Cung cấp analysis cân bằng."
    
    # Dynamic prompt construction
    full_prompt = f"{base_prompt} {instruction}"
    
    prompt_template = ChatPromptTemplate.from_template(full_prompt)
    
    # Dynamic chain based on analysis type
    if analysis_type == "sentiment":
        chain = (
            RunnablePassthrough.assign(type=lambda _: "cảm xúc")
            | prompt_template
            | llm
            | StrOutputParser()
        )
    elif analysis_type == "technical":
        chain = (
            RunnablePassthrough.assign(type=lambda _: "kỹ thuật")
            | prompt_template
            | llm
            | StrOutputParser()
        )
    else:
        chain = (
            RunnablePassthrough.assign(type=lambda _: "tổng quát")
            | prompt_template
            | llm
            | StrOutputParser()
        )
    
    return chain

# Test dynamic chains
print("=== Dynamic Chain Construction ===")

configurations = [
    ("sentiment", "simple"),
    ("technical", "detailed"),
    ("general", "balanced")
]

test_text = "Trí tuệ nhân tạo đang phát triển nhanh chóng và tạo ra nhiều cơ hội mới."

for analysis_type, complexity in configurations:
    print(f"\n🔧 Configuration: {analysis_type} - {complexity}")
    
    # Create dynamic chain
    dynamic_chain = create_dynamic_chain(analysis_type, complexity)
    
    # Execute
    result = dynamic_chain.invoke({"text": test_text})
    
    print(f"📝 Result: {result[:150]}...")
    print("-" * 60)

In [ ]:
# Advanced pattern: Chain with memory/state
class StatefulProcessor:
    """Processor có state để track processed items"""
    
    def __init__(self):
        self.processed_count = 0
        self.processed_items = []
    
    def process(self, input_data):
        """Process input và update state"""
        text = input_data["text"]
        
        # Update state
        self.processed_count += 1
        self.processed_items.append(text[:50] + "..." if len(text) > 50 else text)
        
        # Return processed data với state info
        return {
            "text": text,
            "processing_id": self.processed_count,
            "context": f"This is item #{self.processed_count} processed today"
        }
    
    def get_summary(self):
        return {
            "total_processed": self.processed_count,
            "items": self.processed_items
        }

# Create stateful processor
processor = StatefulProcessor()

# Stateful chain
stateful_chain = (
    RunnableLambda(processor.process)
    | ChatPromptTemplate.from_template(
        "Process the following text (ID: {processing_id}):\n{text}\n\nContext: {context}\n\nSummary:"
    )
    | llm
    | StrOutputParser()
)

print("=== Stateful Chain Testing ===")

# Process multiple items
test_items = [
    "Hôm nay trời đẹp",
    "AI đang thay đổi thế giới",
    "Tôi thích học lập trình"
]

results = []
for item in test_items:
    result = stateful_chain.invoke({"text": item})
    results.append(result)
    print(f"\n📝 Processed: {item}")
    print(f"🔍 Summary: {result[:100]}...")

# Show final state
summary = processor.get_summary()
print(f"\n📊 Final State:")
print(f"Total processed: {summary['total_processed']}")
print(f"Items: {summary['items']}")

## 10. LCEL Best Practices và Performance Tips

In [ ]:
# LCEL Best Practices demonstration
def demonstrate_lcel_best_practices():
    print("=== LCEL BEST PRACTICES ===")
    
    practices = [
        {
            "title": "1. 🔗 USE PIPE OPERATOR",
            "good": "prompt | llm | parser",
            "bad": "parser.parse(llm.invoke(prompt.format(...)))",
            "benefit": "Automatic optimizations, streaming, async support"
        },
        {
            "title": "2. ⚡ LEVERAGE PARALLEL EXECUTION",
            "good": "RunnableParallel({a: chain_a, b: chain_b})",
            "bad": "result_a = chain_a.invoke(); result_b = chain_b.invoke()",
            "benefit": "Significant speedup for independent operations"
        },
        {
            "title": "3. 🛡️ IMPLEMENT FALLBACKS",
            "good": "primary_chain.with_fallbacks([fallback_chain])",
            "bad": "try: primary() except: fallback()",
            "benefit": "Built-in retry logic và error handling"
        },
        {
            "title": "4. 📊 USE PASSTHROUGH WISELY",
            "good": "RunnablePassthrough.assign(new_field=computation)",
            "bad": "Manually merging dictionaries",
            "benefit": "Clean data flow, preserved context"
        },
        {
            "title": "5. 🔄 UTILIZE STREAMING",
            "good": "for chunk in chain.stream(input): ...",
            "bad": "result = chain.invoke(input)  # Wait for complete",
            "benefit": "Better user experience, progressive results"
        }
    ]
    
    for practice in practices:
        print(f"\n{practice['title']}")
        print(f"   ✅ Good: {practice['good']}")
        print(f"   ❌ Bad:  {practice['bad']}")
        print(f"   💡 Benefit: {practice['benefit']}")

demonstrate_lcel_best_practices()

In [ ]:
# Performance optimization examples
def demonstrate_performance_optimizations():
    print("\n=== PERFORMANCE OPTIMIZATION TIPS ===")
    
    tips = [
        {
            "tip": "🚀 Batch Processing",
            "description": "Use .batch() thay vì multiple .invoke() calls",
            "example": "chain.batch([input1, input2, input3])"
        },
        {
            "tip": "⚡ Async When Possible",
            "description": "Use .abatch() cho async batch processing",
            "example": "await chain.abatch(inputs)"
        },
        {
            "tip": "🔄 Smart Parallelization",
            "description": "Group independent operations trong RunnableParallel",
            "example": "RunnableParallel({task1: chain1, task2: chain2})"
        },
        {
            "tip": "💾 Avoid Unnecessary Copies",
            "description": "Use RunnablePassthrough thay vì copying data",
            "example": "RunnablePassthrough.assign(new_field=compute)"
        },
        {
            "tip": "🎯 Minimize LLM Calls",
            "description": "Combine multiple tasks trong single prompt when possible",
            "example": "'Analyze sentiment AND extract keywords: {text}'"
        }
    ]
    
    for tip in tips:
        print(f"\n{tip['tip']}")
        print(f"   📋 {tip['description']}")
        print(f"   💻 Example: {tip['example']}")

demonstrate_performance_optimizations()

print("\n✅ LCEL Basics tutorial completed!")

## Tổng kết

### **LCEL (LangChain Expression Language): Key Benefits**

#### **🔗 Core Features**
- **Pipe Operator (`|`)**: Intuitive chaining syntax
- **Automatic Optimizations**: Built-in performance enhancements
- **Streaming Support**: Progressive output delivery
- **Async Support**: Native async/await capabilities
- **Error Handling**: Built-in retry và fallback mechanisms

#### **🧩 Essential Components**
1. **RunnablePassthrough**: Preserve và enrich data
2. **RunnableParallel**: Execute multiple operations concurrently
3. **RunnableLambda**: Wrap custom functions
4. **RunnableBranch**: Conditional execution logic
5. **RunnableWithFallbacks**: Error recovery patterns

#### **⚡ Performance Advantages**
- **Parallel Execution**: Automatic optimization của independent operations
- **Batch Processing**: Efficient handling của multiple inputs
- **Streaming**: Real-time output delivery
- **Async Support**: Non-blocking execution

#### **🛠️ Advanced Patterns học được**
- **Complex Compositions**: Multi-step workflows
- **Conditional Logic**: Dynamic chain selection
- **State Management**: Stateful processing
- **Error Recovery**: Robust fallback strategies

### **Best Practices**
1. **Always Use Pipe Operator**: Leverage LCEL optimizations
2. **Parallel When Possible**: Group independent operations
3. **Implement Fallbacks**: Handle errors gracefully
4. **Utilize Streaming**: Better user experience
5. **Batch Operations**: Avoid multiple individual calls
6. **Preserve Context**: Use RunnablePassthrough for data flow

### **Next Steps**
- **Sequential Chains**: Multi-step dependent workflows
- **Retrieval Chains**: Integration với vector stores
- **Agent Patterns**: Decision-making workflows
- **Production Deployment**: Scaling và monitoring

LCEL là foundation mạnh mẽ cho building efficient và maintainable LangChain applications!